In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Exercício PySpark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [3]:
spark

In [4]:
df_parquet = spark.read.parquet("dados_cadastro_1.parquet")
df_csv = spark.read.csv("dados_cadastro_2.csv", header=True, sep = "|")
df_json = spark.read.json("dados_cadastro_3.json")

In [5]:
#observando as linhas iniciais de cada df
df_parquet.show(truncate=False)
df_csv.show(truncate=False)
df_json.show(truncate=False)

+--------------+------------------------+---------------+----+------------+------------------------+------+-----------------------+---------------------+------+-------------------+---+-----------------+
|documento_cpf |nome_completo           |data_nascimento|sexo|convenio    |logradouro              |numero|bairro                 |cidade               |pais  |data_cadastro      |uf |__index_level_0__|
+--------------+------------------------+---------------+----+------------+------------------------+------+-----------------------+---------------------+------+-------------------+---+-----------------+
|390.628.415-89|Caroline Rocha          |1982-02-28     |Fem |GoodHealth  |Trevo Santos            |773   |Vila Antena            |Cardoso              |Brasil|1979-10-19T08:09:16|AC |207979           |
|360.847.952-00|Dr. Luiz Otávio Monteiro|2020-09-04     |Masc|Porto Seguro|Trevo Mariana Alves     |184   |Marmiteiros            |Gonçalves            |Brasil|2012-01-22T00:21:53|AC |5181

In [6]:
#observando schema dos df
df_parquet.printSchema()
df_csv.printSchema()
df_json.printSchema()

root
 |-- documento_cpf: string (nullable = true)
 |-- nome_completo: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- convenio: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- data_cadastro: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- __index_level_0__: long (nullable = true)

root
 |-- cpf: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- convenio: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_cadastro: string (nullable = true)
 |-- pais_cadastro: strin

In [7]:
#observando as colunas e quantas sao em cada df
print(df_parquet.columns, len(df_parquet.columns))
print(df_csv.columns, len(df_csv.columns))
print(df_json.columns, len(df_json.columns))

['documento_cpf', 'nome_completo', 'data_nascimento', 'sexo', 'convenio', 'logradouro', 'numero', 'bairro', 'cidade', 'pais', 'data_cadastro', 'uf', '__index_level_0__'] 13
['cpf', 'nome', 'data_nascimento', 'sexo', 'convenio', 'logradouro', 'numero', 'bairro', 'cidade', 'estado', 'data_cadastro', 'pais_cadastro'] 12
['bairro', 'cidade', 'convenio', 'cpf', 'data_cadastro', 'data_nascimento', 'estado', 'logradouro', 'nome', 'numero', 'pais', 'sexo'] 12


In [8]:
#ordem e nomes das colunas que quero usar

ordem_cols = ['nome',
'cpf',
'data_nascimento',
'sexo',
'logradouro',
'numero',
'bairro',
'cidade',
'estado',
'pais',
'convenio',
'data_cadastro']

In [9]:
#padronizando nome das colunas
#parquet
nome_cols_parquet = ['cpf', 'nome', 'data_nascimento', 'sexo', 'convenio', 'logradouro', 'numero', 'bairro', 'cidade', 'pais', 'data_cadastro', 'estado']

#excluindo a coluna index_level_0 do parquet
df_parquet = df_parquet.drop("__index_level_0__")

df_parquet = df_parquet.toDF(*nome_cols_parquet)
df_parquet.show(truncate=False)

+--------------+------------------------+---------------+----+------------+------------------------+------+-----------------------+---------------------+------+-------------------+------+
|cpf           |nome                    |data_nascimento|sexo|convenio    |logradouro              |numero|bairro                 |cidade               |pais  |data_cadastro      |estado|
+--------------+------------------------+---------------+----+------------+------------------------+------+-----------------------+---------------------+------+-------------------+------+
|390.628.415-89|Caroline Rocha          |1982-02-28     |Fem |GoodHealth  |Trevo Santos            |773   |Vila Antena            |Cardoso              |Brasil|1979-10-19T08:09:16|AC    |
|360.847.952-00|Dr. Luiz Otávio Monteiro|2020-09-04     |Masc|Porto Seguro|Trevo Mariana Alves     |184   |Marmiteiros            |Gonçalves            |Brasil|2012-01-22T00:21:53|AC    |
|415.932.076-70|Srta. Emanuelly Caldeira|2011-10-28     |Fem

In [10]:
# csv
df_csv = df_csv.withColumnRenamed("pais_cadastro","pais")
df_csv.show(truncate=False)

+-----------+----------------------+---------------+----+--------+--------------------------+------+-----------------------------+--------------------+------+-------------------+----+
|cpf        |nome                  |data_nascimento|sexo|convenio|logradouro                |numero|bairro                       |cidade              |estado|data_cadastro      |pais|
+-----------+----------------------+---------------+----+--------+--------------------------+------+-----------------------------+--------------------+------+-------------------+----+
|03687145271|Maria Cecília Duarte  |02/11/1989     |Fem |AMIL    |Jardim Rodrigues          |335.0 |São Geraldo                  |Gonçalves           |CE    |1970-07-28 10:05:02|Br  |
|40791586375|Maria Cecília Alves   |10/05/1980     |Fem |AMIL    |Viela de Ribeiro          |651.0 |Goiania                      |Rezende da Praia    |AP    |1992-03-12 07:10:16|Br  |
|52983647056|Laura Cardoso         |02/09/1997     |Fem |AMIL    |Pátio Brenda R

In [11]:
#json
#nome das colunas do json já está correto
df_json.show(truncate=False)

+-----------------------------+-------------------+--------+--------------+-------------+------------------+------------------+-------------------------------+-----------------------+------+----+----+
|bairro                       |cidade             |convenio|cpf           |data_cadastro|data_nascimento   |estado            |logradouro                     |nome                   |numero|pais|sexo|
+-----------------------------+-------------------+--------+--------------+-------------+------------------+------------------+-------------------------------+-----------------------+------+----+----+
|Vila Madre Gertrudes 3ª Seção|Ribeiro Grande     |AMIL    |573.691.208-21|817032248000 |August 26, 2017   |Amapá             |Largo de Viana                 |Ryan Santos            |886   |    |M   |
|Havaí                        |Caldeira das Flores|AMIL    |520.936.741-06|1009317437000|January 30, 2023  |Mato Grosso do Sul|Rodovia Nina Alves             |Juliana Correia        |545   |    |F

In [13]:
# deixando dfs com as colunas na mesma ordem

df_parquet = df_parquet.select(*ordem_cols)
df_parquet.show(truncate=False)

df_csv = df_csv.select(*ordem_cols)
df_csv.show(truncate=False)

df_json = df_json.select(*ordem_cols)
df_json.show(truncate=False)

+------------------------+--------------+---------------+----+------------------------+------+-----------------------+---------------------+------+------+------------+-------------------+
|nome                    |cpf           |data_nascimento|sexo|logradouro              |numero|bairro                 |cidade               |estado|pais  |convenio    |data_cadastro      |
+------------------------+--------------+---------------+----+------------------------+------+-----------------------+---------------------+------+------+------------+-------------------+
|Caroline Rocha          |390.628.415-89|1982-02-28     |Fem |Trevo Santos            |773   |Vila Antena            |Cardoso              |AC    |Brasil|GoodHealth  |1979-10-19T08:09:16|
|Dr. Luiz Otávio Monteiro|360.847.952-00|2020-09-04     |Masc|Trevo Mariana Alves     |184   |Marmiteiros            |Gonçalves            |AC    |Brasil|Porto Seguro|2012-01-22T00:21:53|
|Srta. Emanuelly Caldeira|415.932.076-70|2011-10-28     |Fem

In [17]:
print(df_parquet.dtypes)
print(df_csv.dtypes)
print(df_json.dtypes)

[('nome', 'string'), ('cpf', 'string'), ('data_nascimento', 'string'), ('sexo', 'string'), ('logradouro', 'string'), ('numero', 'string'), ('bairro', 'string'), ('cidade', 'string'), ('estado', 'string'), ('pais', 'string'), ('convenio', 'string'), ('data_cadastro', 'string')]
[('nome', 'string'), ('cpf', 'string'), ('data_nascimento', 'string'), ('sexo', 'string'), ('logradouro', 'string'), ('numero', 'string'), ('bairro', 'string'), ('cidade', 'string'), ('estado', 'string'), ('pais', 'string'), ('convenio', 'string'), ('data_cadastro', 'string')]
[('nome', 'string'), ('cpf', 'string'), ('data_nascimento', 'string'), ('sexo', 'string'), ('logradouro', 'string'), ('numero', 'bigint'), ('bairro', 'string'), ('cidade', 'string'), ('estado', 'string'), ('pais', 'string'), ('convenio', 'string'), ('data_cadastro', 'bigint')]
